In [82]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import timeit
import datetime
import xgboost as xgb
import math
import plotly.express as px
import pandas as pd
import plotly.graph_objects as go
import plotly.io as pio
import plotly.graph_objs as go
import folium
import json
# Offline mode
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)
import joblib
import seaborn as sns
from sklearn.ensemble import IsolationForest
from sklearn.cluster import KMeans
import folium

# libraries for import all file names
from prettytable import PrettyTable
from os import listdir
from os.path import isfile, join
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from sklearn.metrics import accuracy_score , mean_squared_error
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor
from yellowbrick.model_selection import FeatureImportances
from sklearn.linear_model import LinearRegression



In [84]:
#Visualització estacions SMC
m = folium.Map(location=[41.96867, 2.41404], zoom_start=10)

tooltip = ''
folium.Marker(
    [42.07398, 2.20862], popup="<i>CC (Oris)</i>", tooltip=tooltip
).add_to(m)
folium.Marker(
    [42.37717, 2.41456], popup="<i>CG (Molló - Fabert)</i>", tooltip=tooltip
).add_to(m)
folium.Marker(
    [42.25839, 2.36429 ], popup="<i>CI (Sant Pau de Següries)</i>", tooltip=tooltip
).add_to(m)
folium.Marker(
    [41.87813, 2.17873 ], popup="<i>CY (Muntanyola)</i>", tooltip=tooltip
).add_to(m)
folium.Marker(
    [42.39848, 2.15517 ], popup="<i>DG (Nuria)</i>", tooltip=tooltip
).add_to(m)
folium.Marker(
    [42.11653, 2.78969 ], popup="<i>DJ (Banyoles)</i>", tooltip=tooltip
).add_to(m)
folium.Marker(
    [41.96095, 2.63108 ], popup="<i>DN (Anglès)</i>", tooltip=tooltip
).add_to(m)
folium.Marker(
    [41.96867, 2.41404 ], popup="<i>KE (Pantà de Sau)</i>", tooltip=tooltip
).add_to(m)
folium.Marker(
    [42.22189, 2.2427 ], popup="<i>M6 (Sant Joan de les Abadesses)</i>", tooltip=tooltip
).add_to(m)
folium.Marker(
    [42.05398, 3.06195 ], popup="<i>UB ('La Tallada d'Empordà)</i>", tooltip=tooltip
).add_to(m)
folium.Marker(
    [41.87449, 2.92694 ], popup="<i>UN (Cassà de la Selva)</i>", tooltip=tooltip
).add_to(m)
folium.Marker(
    [41.91461, 2.82069 ], popup="<i>UO (Fornells de la Selva)</i>", tooltip=tooltip
).add_to(m)
folium.Marker(
    [41.95224, 2.23271], popup="<i>V3 (Gurb)</i>", tooltip=tooltip
).add_to(m)
folium.Marker(
    [42.11477, 2.21483 ], popup="<i>V4 (Montesquiu)</i>", tooltip=tooltip
).add_to(m)
folium.Marker(
    [41.88244, 2.21483 ], popup="<i>VN (Vilobí d'Onyar)</i>", tooltip=tooltip
).add_to(m)
folium.Marker(
    [41.84008, 2.41877 ], popup="<i>WS (Viladrau)</i>", tooltip=tooltip
).add_to(m)
folium.Marker(
    [42.03947, 2.11993 ], popup="<i>V5 (Perafita)</i>", tooltip=tooltip
).add_to(m)

folium.GeoJson(json.load(open("ter.geojson")), name="Riu Ter").add_to(m)

m

In [45]:
# SMC
Df_SMC_Total = pd.read_excel("finalsDF/DF_SMC.xlsx", index_col=0)
#['CC', 'CG', 'CI', 'CY', 'DG', 'DJ', 'DN', 'KE', 'UB', 'UN', 'UO', 'V3','V4', 'V5', 'VN', 'WS', 'DM_XJ_WF', 'Z4_ZC']

In [48]:
df = Df_SMC_Total
#Crear una versio horaria del dataset
df['timestamp'] = pd.to_datetime(df.index)
df = df.set_index('timestamp').resample("H").mean().reset_index()

In [60]:
#Usem Isolation Forest per trobar anomalies a les precipitacions
output = df[['CC', 'timestamp']]
cc = df[['CC', 'timestamp']]
model=IsolationForest(n_estimators=50, max_samples='auto', contamination=0.005,max_features=1.0)
model.fit(cc[['CC']].values)
output['outliers']=pd.Series(model.predict(cc[['CC']].values)).apply(lambda x: 'yes' if (x == -1) else 'no' )

/var/folders/d4/kxqcphy54q9g3rr5tlcvhv9h0000gn/T/ipykernel_56601/129710627.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [79]:
#Mostrar en un plot les anomalies que trobem amb la precipitació a CC
output = output.loc[output['timestamp'] >= '2020-01-01 00:00:00']
fig = px.scatter(output, x='timestamp', y='CC', color='outliers',title='Outlier CC')
fig.update_xaxes(
    rangeslider_visible=True,
)
fig.show()